Training on Lateral sided images 

IMPORTS

In [1]:


import tarfile
import os
import pickle
from PIL import Image
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from PIL import Image
import tensorflow as tf
import cv2 as cv
from keras.models import load_model
from tensorflow.keras.applications import DenseNet121
from tqdm.notebook import tqdm
from tensorflow.keras.preprocessing.image import img_to_array

from tensorflow import concat
tf.compat.v1.enable_eager_execution()

import cv2
from skimage.transform import resize

from tensorflow.keras.layers import Input, Softmax, RNN, Dense, Embedding, LSTM,Layer,Dropout,GRU
from tensorflow.keras.models import Model
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow import repeat
from sklearn.utils import shuffle
import nltk.translate.bleu_score as bleu
import warnings
from tensorflow.keras.applications.densenet import preprocess_input

warnings.filterwarnings('ignore')


print("done importing")

2024-07-04 11:30:03.465906: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-04 11:30:03.465951: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-04 11:30:03.467882: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-04 11:30:03.478041: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-04 11:30:04.290438: W tensorflow/compiler/tf2

done importing


File which consist image features extracted from chexNet

In [2]:
data.head(5)

NameError: name 'data' is not defined

In [3]:
data.columns

NameError: name 'data' is not defined

0 ===> filename <br>
1 ===> uid <br>
2 ===> findings<br>
3 ===> image_features<br>
4 ===> findings_total<br>
5 ===> dec_ip<br>
6 ===> dec_op<br>

In [5]:
len(data)


6232

In [6]:
X_train, X_test = train_test_split(data.values , test_size = 0.2 )


In [7]:
X_train.shape


(4985, 7)

In [8]:
X_test.shape


(1247, 7)

In [9]:
from tensorflow.keras.preprocessing.text import Tokenizer

t1  = Tokenizer( filters='!"#$%&()*+,-/:;=?@[\\]^_`{|}~\t\n',oov_token='OOV')
t1.fit_on_texts(X_train[:,4])
vocab_size_imp = len(t1.word_index) + 1

dec_inp = t1.texts_to_sequences(X_train[:,5])

dec_inp = pad_sequences(dec_inp, maxlen=76, padding='post')

dec_inp_cv = t1.texts_to_sequences(X_test[:,5])

dec_inp_cv = pad_sequences(dec_inp_cv, maxlen=76, padding='post')

dec_op = t1.texts_to_sequences(X_train[:,6])

dec_op = pad_sequences(dec_op, maxlen=76, padding='post')

dec_op_cv = t1.texts_to_sequences(X_test[:,6])

dec_op_cv = pad_sequences(dec_op_cv, maxlen=76, padding='post')


Vocab Size

In [10]:
vocab_size_imp


2065

In [11]:
type(t1)

keras.src.preprocessing.text.Tokenizer

In [14]:
with open('t2.pickle', 'wb') as handle:
    pickle.dump(t1, handle, protocol=pickle.HIGHEST_PROTOCOL)


In [15]:
t2 = pd.read_pickle('/home/professor/Downloads/fromgit/Deadline/t2.pickle')


In [16]:


imp1 = {}
imp2 = {}
for key,value in t2.word_index.items():
  imp1[value] = key
  imp2[key] = value


In [18]:
import json
with open('imp1.json', 'w') as file1:
    json.dump(imp1, file1)

# Save imp2 to a JSON file
with open('imp2.json', 'w') as file2:
    json.dump(imp2, file2)




##Model Design Of LLM##

In [33]:
@tf.keras.utils.register_keras_serializable()

class Encoder(tf.keras.Model):
    '''
    Encoder model -- That takes a input sequence and returns output sequence
    '''

    def __init__(self,units):
        super().__init__()
        self.units = units
        self.dense = Dense(self.units,name = 'Enc_dense')


    def call(self,img):
       
      '''
          This function takes a sequence input and the initial states of the encoder.
          Pass the input_sequence input to the Embedding layer, Pass the embedding layer ouput to encoder_lstm
          returns -- All encoder_outputs, last time steps hidden and cell state
      '''
      print("ENcoder =======================================")
      #enc_out = self.maxpool(tf.expand_dims(img,axis = 2))
      enc_out = self.dense(img)
      print("IMAGE SHAPE IN ENCODER :- ",img.shape)
      print("ENcoder =======================================END")

      return enc_out


    def initialize_states(self,batch_size):
    
      '''
      Given a batch size it will return intial hidden state and intial cell state.
      If batch size is 32- Hidden state is zeros of size [32,lstm_units], cell state zeros is of size [32,lstm_units]
      '''
      self.batch_size  = batch_size

      self.enc_h =tf.zeros((self.batch_size, self.units))

        
      #self.enc_c = tf.zeros((self.batch_size, self.lstm_size))
      return self.enc_h


In [34]:
@tf.keras.utils.register_keras_serializable()

class Attention(tf.keras.layers.Layer):
  '''
    Class the calculates score based on the scoring_function using Bahdanu attention mechanism.
  '''
  def __init__(self,att_units):
    # Please go through the reference notebook and research paper to complete the scoring functions
    super().__init__()

    self.att_units = att_units

    self.w1 =  tf.keras.layers.Dense( self.att_units , name = 'w1')
    self.w2 =  tf.keras.layers.Dense( self.att_units,name = 'w2')
    self.v =  tf.keras.layers.Dense(1,name = 'v')
      

  def call(self,decoder_hidden_state,encoder_output):
    '''
      Attention mechanism takes two inputs current step -- decoder_hidden_state and all the encoder_outputs.
      * Based on the scoring function we will find the score or similarity between decoder_hidden_state and encoder_output.
        Multiply the score function with your encoder_outputs to get the context vector.
        Function returns context vector and attention weights(softmax - scores)
    '''
    print("Attention ===============================")
    self.decoder_hidden_state = decoder_hidden_state
    '''
       decoder_hidden_state.shape =  (32, 256)
    '''
    self.encoder_output = encoder_output
    '''
        encoder_output.shape =  (32, 256)
    '''
    print("Atten layer")
    print("Dec shape ",decoder_hidden_state.shape, "Enc shape", encoder_output.shape )
      



    self.decoder_hidden_state = tf.expand_dims(self.decoder_hidden_state,axis = 1)
    score = self.v(tf.nn.tanh(
              self.w1(self.decoder_hidden_state) + self.w2(self.encoder_output)))
    attention_weights = tf.nn.softmax(score, axis=1)
    '''
        attention_weights.shape = (32, 32, 1)
    '''
    context_vector = attention_weights * self.encoder_output
    
    context_vector = tf.reduce_sum(context_vector, axis=1)
    '''
        context_vector.shape = (32, 256)
    '''
    print("attention_weights.shape = ", attention_weights.shape)
    print("context_vector.shape = ", context_vector.shape)
    print("encoder_output.shape = ", encoder_output.shape)
    print("decoder_hidden_state.shape = ", decoder_hidden_state.shape)
    
    print("Attention ===============================END")
    return context_vector,attention_weights


In [35]:
@tf.keras.utils.register_keras_serializable()

class OneStepDecoder(tf.keras.Model):
  def __init__(self,tar_vocab_size, embedding_dim, input_length, dec_units  ,att_units):

      # Initialize decoder embedding layer, LSTM and any other objects needed
      super().__init__()
      self.tar_vocab_size = tar_vocab_size
      self.embedding_dim = embedding_dim
      self.input_length = input_length
      self.dec_units = dec_units
      self.att_units = att_units
      self.dec_emb = Embedding(tar_vocab_size,embedding_dim,trainable = True , name = 'dec_embb')
      self.dec_lstm = GRU(self.dec_units, return_state=True, return_sequences=True, name="Decoder_LSTM")
      self.dense   = Dense(self.tar_vocab_size, name = 'one_dec')
      self.attention=Attention( self.att_units)
      self.d1 = Dropout(0.3,name = 'd1')
      self.d2 = Dropout(0.3,name = 'd2')
      self.d3 = Dropout(0.3,name = 'd3')

            

  @tf.function
  def call(self,input_to_decoder, encoder_output, state_h):
    print("ONestepDecoder ================")
    '''
         One step decoder mechanisim step by step:
      A. Pass the input_to_decoder to the embedding layer and then get the output(batch_size,1,embedding_dim)
      B. Using the encoder_output and decoder hidden state, compute the context vector.
      C. Concat the context vector with the step A output
      D. Pass the Step-C output to LSTM/GRU and get the decoder output and states(hidden and cell state)
      E. Pass the decoder output to dense layer(vocab size) and store the result into output.
      F. Return the states from step D, output from Step E, attention weights from Step -B
    '''
    self.input_to_decoder = input_to_decoder
    '''
        input_to_decoder.shape = (32, None)
    '''
    self.encoder_output = encoder_output
    '''
        encoder_output.shape = (32, 256)
    '''
    self.state_h = state_h
    '''
        state_h.shape = (32,256)
    '''
    #A
    target_embedd           = self.dec_emb (self.input_to_decoder)     #(batch_size,1,embedingdim)
    #B
    target_embedd = self.d1(target_embedd)
    '''
        target_embedd.shape = (32, None, 300)
    '''

    context_vector,attention_weights=self.attention(self.state_h,self.encoder_output) #context vector shape = (batch_size,units)
    '''
        context_vector.shape = (32, 256)
        attention_weights.shape = (32, 32, 1)
        
    '''
    #C
    concated = tf.concat([  tf.expand_dims(context_vector, 1),target_embedd], -1)
    concated = self.d2(concated)
    '''
        concated.shape = (32, 1, 556)
    '''

    #D
    lstm_output, hs      = self.dec_lstm(concated, initial_state=self.state_h)
    '''
        lstm_output.shape = (32, 1, 256)
        hs.shape = (32, 256)
    '''
    


    lstm_output = tf.reshape(lstm_output, (-1, lstm_output.shape[2]))
    lstm_output = self.d3(lstm_output)

    '''
        lstm_output.shape = (32, 256)
    '''
    #E
    op = self.dense(lstm_output)
    '''
        op.shape = (32, 2053)
    '''
    #op = tf.squeeze(op,[1])
    
    print("input_to_decoder :- ", input_to_decoder.shape , "state_h :- ", state_h.shape)
    
    print("encoder_output ", state_h.shape, "Target eMBEDING - 2",target_embedd.shape )
    print("Concat :- ", concated.shape)
    print("Lstm output",lstm_output.shape, "OP", op.shape )
    print("HS:- ", hs.shape)
    print("context_vector.shape", context_vector.shape,"attention_weights.shape", attention_weights.shape )
    
    print("ONestepDecoder ================END")

    # print("dec_lstm:- ", dec_lstm)
    return op,hs,attention_weights,context_vector


In [47]:
@tf.keras.utils.register_keras_serializable()

class Decoder(tf.keras.Model):
    '''
        out_vocab_size = 2053
        embedding_dim = 300
        input_length = 76
        dec_units = 256
        att_units = 64
    '''
    def __init__(self,out_vocab_size, embedding_dim, input_length, dec_units ,att_units):
      #Intialize necessary variables and create an object from the class onestepdecoder
      super().__init__()
      self.out_vocab_size = out_vocab_size
      self.embedding_dim = embedding_dim
      self.input_length = input_length
      self.dec_units = dec_units
      self.att_units = att_units
      self.onestep = OneStepDecoder(self.out_vocab_size,self.embedding_dim ,self.input_length,self.dec_units,self.att_units)

        
    @tf.function
    def call(self, input_to_decoder,encoder_output,decoder_hidden_state):


        #Initialize an empty Tensor array, that will store the outputs at each and every time step
        #Create a tensor array as shown in the reference notebook

        #Iterate till the length of the decoder input
            # Call onestepdecoder for each token in decoder_input
            # Store the output in tensorarray
        # Return the tensor array
        print("DECODER =======================")
        all_outputs = tf.TensorArray(tf.float32,size =input_to_decoder.shape[1],name = 'output_arrays' )
        '''
            all_outputs.shape = (32, 76, 2053)
        '''
        self.input_to_decoder = input_to_decoder
        '''
            input_to_decoder.shape = (32, 76)
        '''
        self.encoder_output = encoder_output
        '''
            encoder_output = (32, 256)
        '''
        
        self.decoder_hidden_state = decoder_hidden_state
        '''
            decoder_hidden_state.shape = (32, 256)
        '''

        for timestep in tf.range(input_to_decoder.shape[1]):
          op,hs,attention_weights,context_vector = self.onestep(self.input_to_decoder[:,timestep:timestep+1], 
                                                                self.encoder_output, 
                                                                self.decoder_hidden_state
                                                               )
          self.decoder_hidden_state = hs
          all_outputs = all_outputs.write(timestep,op)
        all_outputs = tf.transpose(all_outputs.stack(),[1,0,2])
        '''
            all_outputs.shape = (32, 76, 2053)
        '''
        '''
            context_vector.shape = (32,256)
        '''
        print("context_vector. shape = ", context_vector.shape)

        print("input_to_decoder" , input_to_decoder.shape)
        print("decoder_hidden_state", decoder_hidden_state.shape)
        print("all_outputs", all_outputs.shape)
        print("encoder_output = ", encoder_output.shape)
        print("DECODER =======================END")
        return all_outputs


In [48]:
@tf.keras.utils.register_keras_serializable()
class encoder_decoder(tf.keras.Model):
  #def __init__(self,#params):
    #Intialize objects from encoder decoder
  
  '''out_vocab_size = 2053
      embedding_size_d = 300
      input_length_d = 76
      lstm_size_d = 256
      att_units = 64
      batch_size = 32
      units = 256
  '''  
  def __init__(self,out_vocab_size , embedding_size_d, input_length_d,lstm_size_d,att_units,batch_size,units):

        #Create encoder object
        #Create decoder object
        #Intialize Dense layer(out_vocab_size) with activation='softmax'

        
        super().__init__()

        self.units = units
        self.out_vocab_size = out_vocab_size
        self.embedding_size_d = embedding_size_d
        self.lstm_size_d = lstm_size_d
        self.input_length_d = input_length_d
        self.batch_size = batch_size
        self.att_units = att_units

        self.encoder = Encoder(self.units)

        self.decoder = Decoder(out_vocab_size , embedding_size_d, input_length_d,lstm_size_d,att_units )
        self.dense   = Dense(self.out_vocab_size,name = 'enc_dec_dense')



  def call(self,data):
    #Intialize encoder states, Pass the encoder_sequence to the embedding layer
    # Decoder initial states are encoder final states, Initialize it accordingly
    # Pass the decoder sequence,encoder_output,decoder states to Decoder
    # return the decoder output
    print("encoder_decoder ===================")
    self.inputs,self.outputs = data[0], data[1]
    '''
        Data[0] value  =  (32,1024)
        Data[1] value  =  (32,76)
        
    '''
    # print("="*20, "ENCODER", "="*20)
    self.encoder_h= self.encoder.initialize_states(self.batch_size)
    self.encoder_output = self.encoder(self.inputs)
    '''
        encoder_output.shape = (32,256)
        encoder_h.shape = (32,256)
    '''
    print("-"*27)
    # print("ENCODER ==> OUTPUT SHAPE",self.encoder_output.shape)
    # print("ENCODER ==> HIDDEN STATE SHAPE",self.encoder_h.shape)
    # print("="*20, "DECODER", "="*20)
    output= self.decoder(self.outputs,self.encoder_output,self.encoder_h)
    '''
        output.shape = (32, 76, 2053)
        
    '''
    # print("FINAL OUTPUT SHAPE",output.shape)
    print("data[0], data[1]  = ", data[0].shape, data[1].shape)
    print("output.shape = ", output.shape)
    print("encoder_decoder =================== END")
    return output


In [49]:
optimizer = tf.keras.optimizers.Adam()
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

def loss_function(real, pred):
  mask = tf.math.logical_not(tf.math.equal(real, 0))
  loss_ = loss_object(real, pred)

  mask = tf.cast(mask, dtype=loss_.dtype)
  loss_ *= mask

  return tf.reduce_mean(loss_)
  #out_vocab_size , embedding_size_d, input_length_d,lstm_size_d,att_units,batch_size)


In [50]:
!mkdir checkpoint


mkdir: cannot create directory ‘checkpoint’: File exists


In [51]:
import os
import tensorflow as tf

# Define current working directory
cwd = os.getcwd()

# Define checkpoint directory and file path
checkpoint_dir = os.path.join(cwd, 'checkpoint')
checkpoint_filepath = os.path.join(checkpoint_dir, 'model_checkpoint.keras')

# Ensure the checkpoint directory exists
os.makedirs(checkpoint_dir, exist_ok=True)

# Create the ModelCheckpoint callback
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=False,
    monitor='val_loss',
    verbose=1,
    mode='min',
    save_best_only=True
)

# Print checkpoint filepath to verify
print("Checkpoint filepath:", checkpoint_filepath)



Checkpoint filepath: /home/professor/Downloads/fromgit/Deadline/checkpoint/model_checkpoint.keras


In [52]:
tf.keras.backend.clear_session()


In [53]:
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.8,mode = 'min',verbose = 1,
                              patience=2, min_lr=0.0001)


In [54]:
model_1 = encoder_decoder(vocab_size_imp,300,76,256,64,32,256)
optimizer = tf.keras.optimizers.Adam()
model_1.compile(optimizer=optimizer,loss=loss_function, metrics = ["accuracy"])


In [55]:
train_inp = np.vstack(X_train[:,3]).astype(float)  # Use np.float64 for double-precision floats
test_inp = np.vstack(X_test[:,3]).astype(float)  # Use np.float32 for single-precision floats (if memory efficiency is a concern)


In [56]:
# Print the shapes of your data
print("train_inp shape:", train_inp.shape)
print("dec_inp shape:", dec_inp.shape)
print("dec_op shape:", dec_op.shape)00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000

print("test_inp shape:", test_inp.shape)
print("dec_inp_cv shape:", dec_inp_cv.shape)
print("dec_op_cv shape:", dec_op_cv.shape)


train_inp shape: (4985, 1024)
dec_inp shape: (4985, 76)
dec_op shape: (4985, 76)
test_inp shape: (1247, 1024)
dec_inp_cv shape: (1247, 76)
dec_op_cv shape: (1247, 76)


model_1.fit([train_inp,dec_inp ],dec_op ,validation_data= ([test_inp, dec_inp_cv],dec_op_cv),batch_size= 32,epochs  = 1,callbacks=[reduce_lr,model_checkpoint_callback] , shuffle=True)


In [57]:
model_1.fit([train_inp,dec_inp ],dec_op ,validation_data= ([test_inp, dec_inp_cv],dec_op_cv),batch_size= 32,epochs  = 1,callbacks=[reduce_lr,model_checkpoint_callback] , shuffle=True)


encoder_decoder ===================
ENcoder =======================================
IMAGE SHAPE IN ENCODER :-  (None, 1024)
ENcoder =======================================END
---------------------------
DECODER =======================
ONestepDecoder ================
Attention ===============================
Atten layer
Dec shape  (32, 256) Enc shape (None, 256)
attention_weights.shape =  (32, None, 1)
context_vector.shape =  (32, 256)
encoder_output.shape =  (None, 256)
decoder_hidden_state.shape =  (32, 256)
Attention ===============================END
input_to_decoder :-  (None, None) state_h :-  (32, 256)
encoder_output  (32, 256) Target eMBEDING - 2 (None, None, 300)
Concat :-  (32, 1, 556)
Lstm output (32, 256) OP (32, 2065)
HS:-  (32, 256)
context_vector.shape (32, 256) attention_weights.shape (32, None, 1)
ONestepDecoder ================END
ONestepDecoder ================
Attention ===============================
Atten layer
Dec shape  (32, 256) Enc shape (None, 256)
attention_w

KeyboardInterrupt: 